In [28]:
import numpy as np
from functools import reduce
from scipy.linalg import eigh
import matplotlib.pyplot as plt
from scipy.sparse import kron, identity, csr_matrix, lil_matrix
from scipy.sparse.linalg import eigsh, eigs
from qutip import Qobj, ptrace
from qutip import commutator as qt_commutator
from tqdm import tqdm
from itertools import product

In [42]:
[2]*3

[2, 2, 2]

In [ ]:
def partial_trace_qubit(rho, keep, dims):
    """Compute the partial trace of a density matrix of qubits."""
    print("keep",keep)
    print("dims",dims)
    keep_dims = np.prod([dims[i] for i in keep])
    print("keep_dims",keep_dims)
    trace_dims = np.prod([dims[i] for i in range(len(dims)) if i not in keep])
    print("trace_dims",trace_dims)
    rho = rho.reshape([keep_dims, trace_dims, keep_dims, trace_dims])
    print("rho",rho)
    return np.trace(rho, axis1=1, axis2=3).reshape([keep_dims, keep_dims])

def vnentropy_qubit(psi, subsystem_size, total_size):

    '''Computes the bipartite entanglement entropy of a pure state.
    
    Parameters:
    psi : np.array
        The wavefunction (state vector) of the full system.
    subsystem_size : int
        The number of qubits in subsystem A.
    total_size : int
        The total number of qubits in the system.
    
    Returns:
    float
        The von Neumann entanglement entropy S_A.'''
    
    psi_matrix =  np.outer(psi, psi.conj())

    # Compute the reduced density matrix rho_A = Tr_B(|psi><psi|)
    rho_A = partial_trace_qubit(psi_matrix, list(range(subsystem_size)), [2]*total_size)  # Partial trace over B
    
    # Compute eigenvalues of rho_A
    eigenvalues = np.linalg.eigvalsh(rho_A)
    
    # Filter out zero eigenvalues to avoid numerical issues in log calculation
    eigenvalues = eigenvalues[eigenvalues > 0]
    
    # Compute von Neumann entropy S_A = -Tr(rho_A log rho_A)
    entropy = -np.sum(eigenvalues * np.log2(eigenvalues))
    
    return entropy

In [ ]:
rho_2b = np.array([[1/2,0,0,1/2],[0,0,0,0],[0,0,0,0],[1/2,0,0,1/2]]) # bell state phi+
print(rho_2b)
print(partial_trace_qubit(rho_2b, [0], [2,2]))
print(partial_trace_qubit(rho_2b, [1], [2,2]))

[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]
[[0.5 0. ]
 [0.  0.5]]
[[0.5 0. ]
 [0.  0.5]]


In [ ]:
# debugging partial trace by comparing numpy vs qutip partial trace - 2 and 3 spins (trace out 1 and 2 spins respectively)

rho_i = 1/np.sqrt(2)
rho_s = np.array([[rho_i**2, rho_i**2], [rho_i**2, rho_i**2]])
rho_2 = np.kron(rho_s, rho_s) # bell state psi+
rho_3 = np.kron(rho_s, rho_2)
rho_4 = np.kron(rho_s, rho_3)
print(f"rho_s: {rho_s}")
print(f"rho_2: {rho_2}")
print(f"rho_3: {rho_3}")
print(f"rho_4: {rho_4}")

rho_s: [[0.5 0.5]
 [0.5 0.5]]
rho_2: [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
rho_3: [[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]]
rho_4: [[0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0

In [31]:
# Trace out the last spin from the 2 spins matrix - 1 spin left

rho_1r = partial_trace_qubit(rho_2, [0], [2, 2])
print(f"rho_1r: {rho_1r}")

rho_2_qobj = Qobj(rho_2, dims=[[2, 2],[2, 2]])
rho_1r_qobj = ptrace(rho_2_qobj, [0])
print(f"rho_1r_qobj: {rho_1r_qobj}")
rho_1r_qobj_dense = rho_1r_qobj.full()
print(f"rho_1r_qobj_dense: {rho_1r_qobj_dense}")

keep [0]
dims [2, 2]
keep_dims 2
trace_dims 2
rho [[[[0.25 0.25]
   [0.25 0.25]]

  [[0.25 0.25]
   [0.25 0.25]]]


 [[[0.25 0.25]
   [0.25 0.25]]

  [[0.25 0.25]
   [0.25 0.25]]]]
rho_1r: [[0.5 0.5]
 [0.5 0.5]]
rho_1r_qobj: Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]
rho_1r_qobj_dense: [[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]


In [ ]:
# Trace out the last spin from the 3 spins matrix - 2 spins left

rho_2rr = partial_trace_qubit(rho_3, [0,1], [2, 2, 2])
print(f"rho_1rr: {rho_2rr}")

rho_3_qobj = Qobj(rho_3, dims=[[2, 2, 2], [2, 2, 2]])
rho_2rr_qobj = ptrace(rho_3_qobj, [0,1])
print(f"rho_1rr_qobj: {rho_2rr_qobj}")
rho_2rr_qobj_dense = rho_2rr_qobj.full()
print(f"rho_1rr_qobj_dense: {rho_2rr_qobj_dense}")

keep [0, 1]
dims [2, 2, 2]
keep_dims 4
trace_dims 2
rho [[[[0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]]

  [[0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]]]


 [[[0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]]

  [[0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]]]


 [[[0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]]

  [[0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]]]


 [[[0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]]

  [[0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]
   [0.125 0.125]]]]
rho_1rr: [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
rho_1rr_bis: [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
rho_1rr_qobj: Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25

In [ ]:
# Trace out the last 2 spins from the 3 spins matrix - 1 spin left

rho_1rr = partial_trace_qubit(rho_3, [0], [2, 2, 2])
print(f"rho_1rr: {rho_1rr}")

rho_3_qobj = Qobj(rho_3, dims=[[2, 2, 2], [2, 2, 2]])
rho_1rr_qobj = ptrace(rho_3_qobj, [0])
print(f"rho_1rr_qobj: {rho_1rr_qobj}")
rho_1rr_qobj_dense = rho_1rr_qobj.full()
print(f"rho_1rr_qobj_dense: {rho_1rr_qobj_dense}")

keep [0]
dims [2, 2, 2]
keep_dims 2
trace_dims 4
rho [[[[0.125 0.125 0.125 0.125]
   [0.125 0.125 0.125 0.125]]

  [[0.125 0.125 0.125 0.125]
   [0.125 0.125 0.125 0.125]]

  [[0.125 0.125 0.125 0.125]
   [0.125 0.125 0.125 0.125]]

  [[0.125 0.125 0.125 0.125]
   [0.125 0.125 0.125 0.125]]]


 [[[0.125 0.125 0.125 0.125]
   [0.125 0.125 0.125 0.125]]

  [[0.125 0.125 0.125 0.125]
   [0.125 0.125 0.125 0.125]]

  [[0.125 0.125 0.125 0.125]
   [0.125 0.125 0.125 0.125]]

  [[0.125 0.125 0.125 0.125]
   [0.125 0.125 0.125 0.125]]]]


ValueError: `source` and `destination` arguments must have the same number of elements

In [34]:
# Trace out the last spin from the 4 spins matrix - 3 spins left

rho_3rrr = partial_trace_qubit(rho_4, [0,1,2], [2, 2, 2, 2])
print(f"rho_3rrr: {rho_3rrr}")

rho_4_qobj = Qobj(rho_4, dims=[[2, 2, 2, 2], [2, 2, 2, 2]])
rho_3rrr_qobj = ptrace(rho_4_qobj, [0,1,2])
print(f"rho_3rrr_qobj: {rho_3rrr_qobj}")
rho_3rrr_qobj_dense = rho_3rrr_qobj.full()
print(f"rho_3rrr_qobj_dense: {rho_3rrr_qobj_dense}")

keep [0, 1, 2]
dims [2, 2, 2, 2]
keep_dims 8
trace_dims 2
rho [[[[0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]]

  [[0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]]]


 [[[0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]]

  [[0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]]]


 [[[0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]]

  [[0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]
   [0.0625 0.0625]]]


 [[[0.06

In [35]:
# Trace out the last 2 spins from the 4 spins matrix - 2 spins left

rho_2rrr = partial_trace_qubit(rho_4, [0,1], [2, 2, 2, 2])
print(f"rho_2rrr: {rho_2rrr}")

rho_4_qobj = Qobj(rho_4, dims=[[2, 2, 2, 2], [2, 2, 2, 2]])
rho_2rrr_qobj = ptrace(rho_4_qobj, [0,1])
print(f"rho_2rrr_qobj: {rho_2rrr_qobj}")
rho_2rrr_qobj_dense = rho_2rrr_qobj.full()
print(f"rho_2rrr_qobj_dense: {rho_2rrr_qobj_dense}")

keep [0, 1]
dims [2, 2, 2, 2]
keep_dims 4
trace_dims 4
rho [[[[0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]]

  [[0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]]

  [[0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]]

  [[0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]]]


 [[[0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]]

  [[0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]]

  [[0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]
   [0.0625 0.0625 0.0625 0.0625]]

 

In [36]:
N=3
print([2]*N)

[2, 2, 2]
